In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

     ------------------------------------ 342.5/342.5 MB 812.1 kB/s eta 0:00:00
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached h5py-2.10.0-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nudenet 2.0.9 requires opencv-python-headless>=4.5.1.48, which is not installed.
torchvision 0.11.3 requires torch==1.10.2, but you have torch 1.12.0 which is incompatible.
streamlit 1.3.0 requires click<8.0,>=7.0, but you have click 8.1.3 which is incompatible.
onnxruntime 1.11.1 requires numpy>=1.21.6, but you have numpy 1.18.5 which is incompatible.
evidently 0.1.56.dev0 requires numpy>=1.19.5, but you have numpy 1.18.5 which is incompatible.
evidently 0.1.56.dev0 requires scipy>=1.5.4, but you have scipy 1.4.1 which is incompatible.


  Using cached scipy-1.4.1-cp37-cp37m-win_amd64.whl (30.9 MB)
  Using cached numpy-1.18.5-cp37-cp37m-win_amd64.whl (12.7 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.2
    Uninstalling scipy-1.7.2:
      Successfully uninstalled scipy-1.7.2
  Attempting uninstall: h5py
    Found existing installation: h5py 3.5.0
    Uninstalling h5py-3.5.0:
      Successfully uninstalled h5py-3.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2

     ---------------------------------------- 52.1/52.1 kB 2.8 MB/s eta 0:00:00


In [2]:
import gym 
import random

In [3]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [5]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:11.0
Episode:2 Score:50.0
Episode:3 Score:19.0
Episode:4 Score:15.0
Episode:5 Score:11.0
Episode:6 Score:29.0
Episode:7 Score:10.0
Episode:8 Score:19.0
Episode:9 Score:27.0
Episode:10 Score:48.0


In [6]:
env.close()

In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [14]:
del model

In [15]:
model = build_model(states, actions)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [12]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [16]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 11:58 - reward: 1.0000

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 60s 6ms/step - reward: 1.0000
98 episodes - episode_reward: 101.663 [10.000, 200.000] - loss: 3.020 - mae: 19.208 - mean_q: 38.875

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 61s 6ms/step - reward: 1.0000
50 episodes - episode_reward: 199.000 [181.000, 200.000] - loss: 7.325 - mae: 39.209 - mean_q: 78.952

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 59s 6ms/step - reward: 1.0000
50 episodes - episode_reward: 199.020 [181.000, 200.000] - loss: 6.590 - mae: 41.305 - mean_q: 83.018

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 1.0000
50 episodes - episode_reward: 199.640 [190.000, 200.000] - loss: 6.959 - mae: 40.472 - mean_q: 81.142

Interval 5 (40000 steps performed)
10000/10000 [==============================] - ETA: 0s - reward: 1.000 - 68s 7ms/step - reward: 1.0000
done, took 314.609 seconds


In [17]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
200.0


In [18]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


In [19]:
env.close()

In [20]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [21]:
del model
del dqn
del env

In [22]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [23]:
dqn.load_weights('dqn_weights.h5f')

In [24]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


In [25]:
env.close()